In [1]:
from Liputan6 import Liputan6 #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [12]:
# health
obj = Liputan6()
end_date = date(2018, 8, 8)
start_date = date(2018, 2, 8)
init_page = 3
list_cat = [['news', 'news'], ['otomotif', 'otomotif'], ['properti', 'properti'],['bisnis','bisnis'],['showbiz','showbiz'],
           ['bola','bola'],['tekno','tekno'],['fashion-beauty','fashion-beauty'],['lifestye','lifestye']]
init_cat = list_cat[6:]

In [ ]:
for single_date in daterange(start_date, end_date):
    for cat in init_cat:
        clear_output()
        print(single_date.strftime("%Y-%m-%d"))
        print(cat[0])
        obj.getAllBerita([], init_page, cat[0], cat[1], single_date.strftime("%Y/%m/%d"))
        init_page = 1
    init_cat = list_cat

2018-02-09
news
page  1
https://www.liputan6.com/news/indeks/2018/02/09?page=1
Insert berita  Top 3 News Hari Ini: Drama Perpisahan Bocah Argentina dengan Ayah yang Menculiknya
masuk
Insert berita  Sabu 1 Ton Ditemukan dalam Kapal Sunrise Glory di Batam
masuk
Insert berita  PNS Pemkot Jaksel Jadi Tersangka, Ini Sikap Gubenur Anies
masuk
Insert berita  AJI dan IJTI Desak Revisi Tanggal Hari Pers Nasional
masuk
Insert berita  Hari Pers Nasional, Mendes Eko Apresiasi Media Publikasikan Kemajuan Desa
masuk
Insert berita  Selain Jawa Barat, Ini 4 Daerah Rawan Saat Pilkada Serentak
masuk
Insert berita  Ketua DPR: Maraknya Hoax Jadi Tantangan Media Mainstream
masuk
Insert berita  Cegah Insiden Pelawak di Hong Kong, PASKI-Kemlu Akan Sosialisasi Visa Kerja
masuk
Insert berita  Kalapas Bekasi Tegaskan Tak Ada Sipir Terlibat Kasus Prostitusi Online
masuk
Insert berita  Polisi Sebut Tersangka Crane Jatuh di Jatinegara Lalai
masuk
Insert berita  Anies Janji Mudahkan Pengurusan Surat Hilang karena B